In [33]:
import pandas as pd
import numpy as np
import re
import xlsxwriter

In [34]:
def clean_data(data):
    data = data.iloc[7:]
    data = data.iloc[:, 1:]
    data = data.reset_index(drop=True)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
    return data

In [35]:
ce1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\CE_PlantaContrata_1erSemestre.xlsx')
ce2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\CE_PlantaContrata_2doSemestre.xlsx')
ptu1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\PTU_PLANTACONTRATA_1erSemestre.xlsx')
ptu2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\PTU_PLANTACONTRATA_2doSemestre.xlsx')
tns1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\TNS_PLANTACONTRATA_1erSemestre.xlsx')
tns2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\TNS_PLANTACONTRATA_2doSemestre.xlsx')

In [36]:
ce1 = clean_data(ce1)
ce2 = clean_data(ce2)
ptu1 = clean_data(ptu1)
ptu2 = clean_data(ptu2)
tns1 = clean_data(tns1)
tns2 = clean_data(tns2)

In [37]:
agrupacion_coberturas = ['proceso','semestre','sede','tipo_carrera','facultad','codigo_carrera','nombre_carrera','rut_completo','nombres','apaterno','amaterno','contrato','estado']
agrupacion_resultados = ['proceso','semestre','sede','tipo_carrera','facultad','codigo_carrera','nombre_carrera','rut_completo','nombres','apaterno','amaterno','contrato']
estados_sinpendiente = ['finalizado','no_iniciada']
estados = ['finalizado','no_iniciada','pendiente']
nuevos_nombres = {'facultad':'unidad_academica','nombres':'nombre_docente','apaterno':'apaterno_docente','amaterno':'amaterno_docente','rut_completo':'rut_docente'}
escala_puntuacion = {'Siempre': 3.0, 'Generalmente': 2.0, 'Pocas veces': 1.0, 'Nunca': 0.0, '--': 0.0}
columnas_excluidas = ['titulo','grado','sexo','edad','jerarquia','tipo_jornada','inicio_aplicacion','fin_aplicacion','fecha_inicio','fecha_fin']
agrupacion_merge = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','rut_docente','nombre_docente','apaterno_docente','amaterno_docente','contrato']
estado_finalizado = ['finalizado']

In [38]:
def coberturas_autoevaluacion(data):
    try:
        data_coberturas = data.groupby(agrupacion_coberturas).size().reset_index(name='count')
        data_coberturas = data_coberturas.pivot_table(index=agrupacion_resultados, columns='estado', values='count', fill_value=0).reset_index()
        data_coberturas = data_coberturas.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
        data_coberturas = data_coberturas.rename(columns=nuevos_nombres)
        data_coberturas['total'] = data_coberturas[estados].sum(axis=1)
        data_coberturas['finalizado_percent'] = (data_coberturas['finalizado'] / data_coberturas['total']) * 100
    except KeyError:
        try:
            data_coberturas = data.groupby(agrupacion_coberturas).size().reset_index(name='count')
            data_coberturas = data_coberturas.pivot_table(index=agrupacion_resultados, columns='estado', values='count', fill_value=0).reset_index()
            data_coberturas = data_coberturas.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
            data_coberturas = data_coberturas.rename(columns=nuevos_nombres)
            data_coberturas['total'] = data_coberturas[estados_sinpendiente].sum(axis=1)
            data_coberturas['finalizado_percent'] = (data_coberturas['finalizado'] / data_coberturas['total']) * 100
        except:
            data_coberturas = data.groupby(agrupacion_coberturas).size().reset_index(name='count')
            data_coberturas = data_coberturas.pivot_table(index=agrupacion_resultados, columns='estado', values='count', fill_value=0).reset_index()
            data_coberturas = data_coberturas.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
            data_coberturas = data_coberturas.rename(columns=nuevos_nombres)
            data_coberturas['total'] = data_coberturas[estado_finalizado].sum(axis=1)
            data_coberturas['finalizado_percent'] = (data_coberturas['finalizado'] / data_coberturas['total']) * 100
    
    
    return data_coberturas

In [39]:
def resultados_autoevaluacion(data):
    
    columnas_respuesta = [col for col in data.columns if col.startswith('respuesta')]
    data_resultados = data[(data['estado']=="Finalizado")]
    data_resultados[columnas_respuesta] = data_resultados[columnas_respuesta].replace(escala_puntuacion)
    data_resultados = data_resultados.drop(columns=columnas_excluidas)
    data_resultados = data_resultados.groupby(agrupacion_resultados).mean().round(2).reset_index()
    data_resultados = data_resultados.rename(columns=nuevos_nombres)

    return data_resultados

In [40]:
ce1_coberturas = coberturas_autoevaluacion(ce1)
ce2_coberturas = coberturas_autoevaluacion(ce2)
ptu1_coberturas = coberturas_autoevaluacion(ptu1)
ptu2_coberturas = coberturas_autoevaluacion(ptu2)
tns1_coberturas = coberturas_autoevaluacion(tns1)
tns2_coberturas = coberturas_autoevaluacion(tns2)

In [ ]:
ce1_resultados = resultados_autoevaluacion(ce1)
ce2_resultados = resultados_autoevaluacion(ce2)
ptu1_resultados = resultados_autoevaluacion(ptu1)
ptu2_resultados = resultados_autoevaluacion(ptu2)
tns1_resultados = resultados_autoevaluacion(tns1)
tns2_resultados = resultados_autoevaluacion(tns2)

In [42]:
Coberturas_Autoevaluacion = pd.concat([ce1_coberturas,ce2_coberturas,ptu1_coberturas,ptu2_coberturas,tns1_coberturas,tns2_coberturas])
Resultados_Autoevaluacion = pd.concat([ce1_resultados,ce2_resultados,ptu1_resultados,ptu2_resultados,tns1_resultados,tns2_resultados])

In [43]:
autoevaluacion_2022 = Coberturas_Autoevaluacion.merge(
    right = Resultados_Autoevaluacion,
    how = "outer",
    on = agrupacion_merge
)

In [44]:
columnas_respuesta = [col for col in autoevaluacion_2022.columns if col.startswith('respuesta')]

In [45]:
autoevaluacion_2022[columnas_respuesta] = autoevaluacion_2022[columnas_respuesta].fillna(0)

In [47]:
autoevaluacion_2022['pendiente'] = autoevaluacion_2022['pendiente'].fillna(0)

In [48]:
autoevaluacion_2022['no_iniciada'] = autoevaluacion_2022['no_iniciada'].fillna(0)

In [49]:
columnas_respuesta = [col for col in autoevaluacion_2022.columns if col.startswith('respuesta')]
respuestas_mediador = [col for col in autoevaluacion_2022.columns if col.startswith('respuesta_1_')]
respuestas_indagador = [col for col in autoevaluacion_2022.columns if col.startswith('respuesta_2_')]
respuestas_gestion = [col for col in autoevaluacion_2022.columns if col.startswith('respuesta_3_')]

In [50]:
autoevaluacion_2022[respuestas_mediador].mean(axis=1).round(2)
autoevaluacion_2022[respuestas_indagador].mean(axis=1).round(2)
autoevaluacion_2022[respuestas_gestion].mean(axis=1).round(2)

0      2.0
1      3.0
2      3.0
3      3.0
4      3.0
      ... 
745    0.0
746    3.0
747    3.0
748    3.0
749    3.0
Length: 750, dtype: float64